In [1]:
import pandas as pd
import numpy as np
import matplotlib
import torch
from ecog_speech import datasets, feature_processing, experiments, pipeline
from ecog_speech.models import chang2020 as chm
from tqdm.auto import tqdm

%matplotlib inline

In [2]:
hvs_cls = datasets.BaseDataset.get_dataset_by_name('hvs')

hvs_tuples = hvs_cls.make_tuples_from_sets_str('UCSD-4')

hvs = hvs_cls(hvs_tuples, initialize_data=False)

2022-07-05 18:01:22,256 - ecog_speech.datasets.HarvardSentences.__attrs_post_init__:337 - DEBUG - preparing pipeline
2022-07-05 18:01:22,257 - ecog_speech.datasets.HarvardSentences.__attrs_post_init__:340 - DEBUG - Available pipelines: ['random_sample', 'audio_gate', 'region_classification', 'region_classification_from_word_stim', 'audio_gate_speaking_only', 'default']


In [5]:
ucsd_data_map = hvs_cls.load_data(*hvs_tuples[0])#('UCSD', 4)

2022-07-05 18:02:16,326 - ecog_speech.datasets.HarvardSentences.load_data:788 - INFO - -----------Subset: None------------
2022-07-05 18:02:16,327 - ecog_speech.datasets.HarvardSentences.load_data:789 - INFO - ---4-1-1-UCSD---
2022-07-05 18:02:16,327 - ecog_speech.datasets.HarvardSentences.make_filename:1221 - INFO - Harvard sentences only uses location and patient identifiers
2022-07-05 18:02:16,328 - ecog_speech.datasets.HarvardSentences.load_data:792 - DEBUG - Path : /home/morgan/Projects/CMSCResearch/ECOGSpeech/ecog_speech/../data/HarvardSentences/UCSD/Data/UCSD04_Task_1.mat
2022-07-05 18:02:16,476 - ecog_speech.datasets.HarvardSentences.load_mat_from_path:757 - INFO - Couldn't load UCSD04_Task_1.mat with scipy (vers > 7.3?) - using package 'mat73' to load
2022-07-05 18:02:36,912 - ecog_speech.datasets.HarvardSentences.load_data:795 - DEBUG - Matlab keys : ['EKG_signal', 'Labels', 'audio', 'fs_audio', 'fs_signal', 'label_contact_common', 'label_contact_r_a_s', 'sEEG_signal', 'start

In [7]:
from sklearn.pipeline import Pipeline
from sklearn import set_config
set_config(display="diagram")

In [10]:
t_pipeline = hvs.pipeline_map['region_classification']
t_pipeline

Pipeline(steps=[('parse_signal',
                 ParseTimeSeriesArrToFrame(array_key='sEEG_signal', default_fs=1200, fs_key='fs_signal', output_key='signal')),
                ('parse_audio',
                 ParseTimeSeriesArrToFrame(array_key='audio', default_fs=48000, fs_key='fs_audio', reshape=-1)),
                ('parse_stim',
                 ParseTimeSeriesArrToFrame(array_key='stimcode', default_fs=1200, fs_key='fs_signal', output_k...
                 WindowSampleIndicesFromStim(sample_n=1000, stim_key='mouthing_region_stim', stim_value_remap=2, target_offset_shift=Timedelta('-1 days +23:59:59.900000'), target_onset_shift=Timedelta('0 days 00:00:00.100000'))),
                ('imagining_indices',
                 WindowSampleIndicesFromStim(sample_n=1000, stim_key='imagining_region_stim', stim_value_remap=3, target_offset_shift=Timedelta('-1 days +23:59:59.900000'), target_onset_shift=Timedelta('0 days 00:00:00.100000'))),
                ('output', 'passthrough')])

In [12]:
region_data_map = t_pipeline.transform(ucsd_data_map)

2022-07-05 18:05:26,335 - ecog_speech.pipeline.ParseTimeSeriesArrToFrame.process:45 - DEBUG - Accessing fs_signal
2022-07-05 18:05:26,336 - ecog_speech.pipeline.ParseTimeSeriesArrToFrame.process:52 - DEBUG - Input source frequency, fs object: 1024.0
2022-07-05 18:05:26,361 - ecog_speech.pipeline.ParseTimeSeriesArrToFrame.process:71 - INFO - sEEG_signal@1024, shape: (1050000, 90), [(Timedelta('0 days 00:00:00'), Timedelta('0 days 00:17:05.389648438'))]
2022-07-05 18:05:26,416 - ecog_speech.pipeline.ParseTimeSeriesArrToFrame.transform:22 - DEBUG - Updated keys: {'fs_signal', 'signal'}
2022-07-05 18:05:26,417 - ecog_speech.pipeline.ParseTimeSeriesArrToFrame.process:45 - DEBUG - Accessing fs_audio
2022-07-05 18:05:26,417 - ecog_speech.pipeline.ParseTimeSeriesArrToFrame.process:52 - DEBUG - Input source frequency, fs object: 48000.0
2022-07-05 18:05:37,492 - ecog_speech.pipeline.ParseTimeSeriesArrToFrame.process:71 - INFO - audio@48000, shape: (49218750,), [(Timedelta('0 days 00:00:00'), Ti

Processing stim regions:   0%|          | 0/25 [00:00<?, ?it/s]

2022-07-05 18:06:10,775 - ecog_speech.pipeline.WindowSampleIndicesFromStim.process:1167 - INFO - Number of samples keys in sample index: {25000: 1}
2022-07-05 18:06:10,776 - ecog_speech.pipeline.WindowSampleIndicesFromStim.transform:22 - DEBUG - Updated keys: {'index_source_map', 'n_samples_per_window', 'sample_index_map'}
2022-07-05 18:06:10,776 - ecog_speech.pipeline.WindowSampleIndicesFromStim.process:1104 - INFO - (512, Timedelta('0 days 00:00:00.500000'))
2022-07-05 18:06:10,776 - ecog_speech.pipeline.WindowSampleIndicesFromStim.process:1105 - INFO - Samples per window: 256


Processing stim regions:   0%|          | 0/25 [00:00<?, ?it/s]

2022-07-05 18:06:15,390 - ecog_speech.pipeline.WindowSampleIndicesFromStim.process:1167 - INFO - Number of samples keys in sample index: {25000: 1}
2022-07-05 18:06:15,391 - ecog_speech.pipeline.WindowSampleIndicesFromStim.transform:22 - DEBUG - Updated keys: {'index_source_map', 'n_samples_per_window', 'sample_index_map'}
2022-07-05 18:06:15,391 - ecog_speech.pipeline.WindowSampleIndicesFromStim.process:1104 - INFO - (512, Timedelta('0 days 00:00:00.500000'))
2022-07-05 18:06:15,391 - ecog_speech.pipeline.WindowSampleIndicesFromStim.process:1105 - INFO - Samples per window: 256


Processing stim regions:   0%|          | 0/25 [00:00<?, ?it/s]

2022-07-05 18:06:20,067 - ecog_speech.pipeline.WindowSampleIndicesFromStim.process:1167 - INFO - Number of samples keys in sample index: {25000: 1}
2022-07-05 18:06:20,068 - ecog_speech.pipeline.WindowSampleIndicesFromStim.transform:22 - DEBUG - Updated keys: {'index_source_map', 'n_samples_per_window', 'sample_index_map'}
2022-07-05 18:06:20,068 - ecog_speech.pipeline.WindowSampleIndicesFromStim.process:1104 - INFO - (512, Timedelta('0 days 00:00:00.500000'))
2022-07-05 18:06:20,069 - ecog_speech.pipeline.WindowSampleIndicesFromStim.process:1105 - INFO - Samples per window: 256


Processing stim regions:   0%|          | 0/25 [00:00<?, ?it/s]

2022-07-05 18:06:24,729 - ecog_speech.pipeline.WindowSampleIndicesFromStim.process:1167 - INFO - Number of samples keys in sample index: {25000: 1}
2022-07-05 18:06:24,730 - ecog_speech.pipeline.WindowSampleIndicesFromStim.transform:22 - DEBUG - Updated keys: {'index_source_map', 'n_samples_per_window', 'sample_index_map'}


In [13]:
region_data_map.keys()

dict_keys(['EKG_signal', 'Labels', 'audio', 'fs_audio', 'fs_signal', 'label_contact_common', 'label_contact_r_a_s', 'sEEG_signal', 'start_stop_word_ms', 'stimcode', 'signal', 'stim', 'sensor_ras_df', 'sensor_ras_coord_arr', 'audio_mel_spec', 'good_sensor_columns', 'bad_sensor_columns', 'word_start_stop_times', 'sent_start_stop_time', 'speaking_word_stim', 'speaking_word_stim_mask', 'listening_word_stim', 'listening_word_stim_mask', 'mouthing_word_stim', 'mouthing_word_stim_mask', 'imagining_word_stim', 'imagining_word_stim_mask', 'speaking_region_stim', 'speaking_region_stim_mask', 'listening_region_stim', 'listening_region_stim_mask', 'mouthing_region_stim', 'mouthing_region_stim_mask', 'imagining_region_stim', 'imagining_region_stim_mask', 'sample_index_map', 'n_samples_per_window', 'index_source_map'])

### Example Dev: modifying or extending an existing pipelnie
- Requires a passthrough output, so append to the steps that we want to pull out

In [16]:
t_pipe = Pipeline(hvs.pipeline_map['audio_gate'].steps[:8] + [('output', 'passthrough')])
t_pipe

Pipeline(steps=[('parse_signal',
                 ParseTimeSeriesArrToFrame(array_key='sEEG_signal', default_fs=1200, fs_key='fs_signal', output_key='signal')),
                ('parse_audio',
                 ParseTimeSeriesArrToFrame(array_key='audio', default_fs=48000, fs_key='fs_audio', reshape=-1)),
                ('parse_stim',
                 ParseTimeSeriesArrToFrame(array_key='stimcode', default_fs=1200, fs_key='fs_signal', output_key='stim', reshape=-1)),
                ('parse_sensor_ras', ParseSensorRAS()),
                ('extract_mfc', ExtractMFCC()),
                ('sensor_selection', IdentifyGoodAndBadSensors()),
                ('rescale_signal', StandardNormSignal()),
                ('subsample', SubsampleSignal()), ('output', 'passthrough')])